# Bioinformatik Projekt: Otsu Thresholding und Bimodalitäts-Verbesserung

Dieses Jupyter Notebook demonstriert die Funktionsweise unseres Bildverarbeitungsprojekts, bei dem wir verschiedene Varianten des Otsu-Verfahrens zur automatischen Schwellenwertbestimmung auf Mikroskopbilder anwenden und evaluieren.

## 🔧 Preprocessing-Modul: Bimodalitätsverstärkung

Die Funktion `enhance_bimodality` verbessert die Trennung zwischen Vorder- und Hintergrundintensitäten. Sie kombiniert:
- adaptive Gamma-Korrektur (abhängig von der mittleren Helligkeit)
- Histogramm-Equalisierung über die CDF

Dies erleichtert die spätere Segmentierung mittels Otsu-Schwellenwerten.


In [ ]:
# src/bimodality_enhancement.py
import numpy as np

def enhance_bimodality(img: np.ndarray) -> np.ndarray:
    """
    Enhances the bimodality of a grayscale image by applying adaptive gamma correction
    followed by histogram equalization.
    """
    if img.dtype != np.float32 and img.max() > 1.0:
        img = img / 255.0

    mean_intensity = np.mean(img)
    gamma = 3.0 if mean_intensity >= 0.5 else 0.5
    img_gamma = np.power(img, gamma)

    img_8bit = (img_gamma * 255).astype(np.uint8)
    hist, _ = np.histogram(img_8bit.flatten(), bins=256, range=[0, 256])
    cdf = hist.cumsum()
    cdf_normalized = cdf * 255 / cdf[-1]

    img_eq = cdf_normalized[img_8bit].astype(np.uint8)
    return img_eq


## 📉 Otsu Thresholding (Skimage-kompatibel)

Der folgende Abschnitt zeigt eine eigene Reimplementierung des Otsu-Verfahrens zur globalen Schwellenwertbestimmung. Die Funktion `otsu_threshold_skimage_like` imitiert exakt das Verhalten von `skimage.filters.threshold_otsu`, inklusive Histogramm-Normalisierung, Schwellenwertberechnung und Rückprojektion auf den Originalwertebereich.

- `custom_histogram` sorgt für kompatible Histogramme bei beliebigen Intensitätsbereichen.
- Das Verfahren maximiert die Varianz zwischen zwei Klassen (Hintergrund vs. Objekt).


In [ ]:
# src/Complete_Otsu_Global.py

import numpy as np
from typing import Tuple

def custom_histogram(image: np.ndarray, nbins: int = 256) -> Tuple[np.ndarray, np.ndarray]:
    img_min, img_max = image.min(), image.max()
    image_scaled = (image - img_min) / (img_max - img_min) * 255
    hist, bin_edges = np.histogram(image_scaled.ravel(), bins=nbins, range=(0, 255))
    bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2
    return hist, bin_centers

def otsu_threshold_skimage_like(image: np.ndarray) -> float:
    hist, bin_centers = custom_histogram(image, nbins=256)
    p = hist.astype(np.float64) / hist.sum()

    omega0 = np.cumsum(p)
    omega1 = np.cumsum(p[::-1])[::-1]
    mu0 = np.cumsum(p * bin_centers)
    mu1 = np.cumsum((p * bin_centers)[::-1])[::-1]

    sigma_b_squared = (omega0[:-1] * omega1[1:] * (mu0[:-1] / omega0[:-1] - mu1[1:] / omega1[1:])**2)
    t_idx = np.argmax(sigma_b_squared)
    t_scaled = bin_centers[t_idx]

    img_min, img_max = image.min(), image.max()
    t_original = t_scaled / 255 * (img_max - img_min) + img_min
    return t_original


## 📏 Evaluation: Dice Score

Zur quantitativen Bewertung unserer Segmentierungsergebnisse verwenden wir den **Dice Similarity Coefficient (DSC)**, der die Überlappung zweier binärer Masken misst. Ein Wert von:
- `1.0` bedeutet perfekte Übereinstimmung,
- `0.0` bedeutet keinerlei Überlappung.

Die Funktion `dice_score` vergleicht dabei unser vorhergesagtes Binärbild mit einem Ground-Truth-Maskenbild.


In [ ]:
# src/Dice_Score.py

import numpy as np

def dice_score(prediction: np.ndarray, ground_truth: np.ndarray) -> float:
    """
    Computes the Dice coefficient (DSC) between two binary masks.
    """
    if prediction.shape != ground_truth.shape:
        raise ValueError("Prediction and ground truth images must have the same shape.")

    sum_prediction = np.sum(prediction)
    sum_ground_truth = np.sum(ground_truth)
    positive_overlap = np.sum(np.logical_and(prediction, ground_truth))

    if sum_prediction + sum_ground_truth == 0:
        return 1.0

    return 2 * positive_overlap / (sum_prediction + sum_ground_truth)


## 🖼️ Bildauswahl und -einlesung

Die Funktion `find_and_load_image` ermöglicht es, beliebige Beispielbilder rekursiv im Projektordner `data-git` zu finden und als Graustufenbild zu laden. Das Bild wird als `numpy.ndarray` zurückgegeben und ist bereit zur Verarbeitung.


In [ ]:
# utility: Bild rekursiv finden und laden
from pathlib import Path
from skimage import io
import numpy as np

def find_and_load_image(filename: str,
                        data_root: str = "data-git",
                        as_gray: bool = True) -> np.ndarray:
    """
    Findet und lädt ein Bild rekursiv aus dem data_root-Ordner.
    """
    root = Path(data_root)
    for candidate in root.rglob(filename):
        return io.imread(str(candidate), as_gray=as_gray)
    raise FileNotFoundError(f"{filename!r} nicht gefunden unter {data_root!r}")

# Beispiel (ausführbar nur im echten Projektverzeichnis):
# img = find_and_load_image("t01.tif")
# print("Shape:", img.shape, "Min/Max:", img.min(), img.max())


## 📊 Histogrammanalyse

Zur Analyse der Intensitätsverteilung eines Bildes nutzen wir `compute_gray_histogram` und `plot_gray_histogram`. Diese Funktionen helfen, den Kontrast und die Bimodalität zu beurteilen — wichtige Voraussetzungen für eine effektive Otsu-Segmentierung.


In [ ]:
# src/gray_hist.py

import numpy as np
from matplotlib import pyplot as plt
from PIL import Image
from pathlib import Path
from typing import Union, Tuple

def compute_gray_histogram(
    image_source: Union[Path, str, np.ndarray],
    bins: int = 256,
    value_range: Tuple[int, int] = (0, 255)
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Computes histogram of grayscale values from image path or array.
    """
    if isinstance(image_source, (Path, str)):
        img = Image.open(str(image_source)).convert("L")
        arr = np.array(img)
    elif isinstance(image_source, np.ndarray):
        arr = image_source
    else:
        raise TypeError("Expected file path or NumPy array.")

    hist, bin_edges = np.histogram(arr.ravel(), bins=bins, range=value_range)
    return hist, bin_edges

def plot_gray_histogram(hist: np.ndarray, bin_edges: np.ndarray):
    """
    Plots a grayscale histogram.
    """
    plt.figure(figsize=(8, 4))
    plt.bar(bin_edges[:-1], hist, width=bin_edges[1] - bin_edges[0], align='edge')
    plt.xlabel("Pixel intensity")
    plt.ylabel("Frequency")
    plt.title("Grayscale Histogram")
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


## 📂 Laden der Datensätze

Die folgenden Funktionen ermöglichen das strukturierte Laden von Bildern und zugehörigen Ground-Truth-Masken für verschiedene Datensätze:

- `N2DH-GOWT1` (2D-HeLa-Zellen, .tif)
- `N2DL-HeLa` (2D-HeLa-Zellen, .tif)
- `NIH3T3` (Maus-Fibroblasten, .png)

Alle Funktionen geben vier Listen zurück:
1. Bilder (`imgs`)
2. Ground Truths (`gts`)
3. Bildpfade (`img_paths`)
4. GT-Pfade (`gt_paths`)

So wird konsistente Zuordnung garantiert.


In [ ]:
# utility: Dataset Loader
import os
from skimage.io import imread
from glob import glob
import sys

script_dir = os.path.dirname(os.path.realpath(__file__))
project_root = os.path.abspath(os.path.join(script_dir, ".."))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

def load_n2dh_gowt1_images(base_path=None):
    if base_path is None:
        base_path = os.path.join(project_root, "data-git", "N2DH-GOWT1")
    img_dir, gt_dir = os.path.join(base_path, "img"), os.path.join(base_path, "gt")
    img_paths = sorted(glob(os.path.join(img_dir, "*.tif")))
    gt_paths  = sorted(glob(os.path.join(gt_dir, "*.tif")))
    imgs = [imread(p, as_gray=True) for p in img_paths]
    gts  = [imread(p, as_gray=True) for p in gt_paths]
    return imgs, gts, img_paths, gt_paths

def load_n2dl_hela_images(base_path=None):
    if base_path is None:
        base_path = os.path.join(project_root, "data-git", "N2DL-HeLa")
    img_dir, gt_dir = os.path.join(base_path, "img"), os.path.join(base_path, "gt")
    img_paths = sorted(glob(os.path.join(img_dir, "*.tif")))
    gt_paths  = sorted(glob(os.path.join(gt_dir, "*.tif")))
    imgs = [imread(p, as_gray=True) for p in img_paths]
    gts  = [imread(p, as_gray=True) for p in gt_paths]
    return imgs, gts, img_paths, gt_paths

def load_nih3t3_images(base_path=None):
    if base_path is None:
        base_path = os.path.join(project_root, "data-git", "NIH3T3")
    img_dir, gt_dir = os.path.join(base_path, "img"), os.path.join(base_path, "gt")
    img_paths = sorted(glob(os.path.join(img_dir, "*.png")))
    gt_paths  = sorted(glob(os.path.join(gt_dir, "*.png")))
    imgs = [imread(p, as_gray=True) for p in img_paths]
    gts  = [imread(p, as_gray=True) for p in gt_paths]
    return imgs, gts, img_paths, gt_paths


## 🧠 Lokales Otsu-Thresholding

Im Gegensatz zur globalen Methode berechnet `local_otsu` für jedes Pixel einen **lokalen Schwellenwert**, basierend auf einem Fenster um den Pixel herum. Dies ist besonders nützlich bei Bildern mit ungleichmäßiger Ausleuchtung oder variabler Hintergrundintensität.

### Methode:
- Gleitendes Fenster mit konfigurierbarem Radius (Standard: 15)
- Für jedes Fenster wird die Otsu-Schwelle mit unserer `otsu_threshold_skimage_like` Methode berechnet
- Bild wird „reflektierend“ gepolstert, um Randverluste zu vermeiden


In [ ]:
# src/local_otsu.py

import numpy as np
import os
import sys

script_dir = os.path.dirname(os.path.realpath(__file__))
project_root = os.path.abspath(os.path.join(script_dir, ".."))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.Complete_Otsu_Global import otsu_threshold_skimage_like

def local_otsu(image: np.ndarray, radius: int = 15) -> np.ndarray:
    """
    Computes local Otsu threshold map using sliding window approach.
    """
    H, W = image.shape
    t_map = np.zeros((H, W), dtype=image.dtype)
    pad = radius
    padded = np.pad(image, pad, mode="reflect")
    w = 2 * radius + 1

    for i in range(H):
        if i % 50 == 0 or i == H - 1:
            print(f"Processing row {i+1}/{H}...")
        for j in range(W):
            block = padded[i : i + w, j : j + w]
            t = otsu_threshold_skimage_like(block)
            t_map[i, j] = t

    return t_map
